In [ ]:
#loading the packages
from transformers import pipeline
import transformers
import pandas as pd
import re, os
from tqdm import tqdm
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
#Loading the model trained for token classification using Stackover flow Text mixed with codes
pipe = pipeline("token-classification", model="mrm8488/codebert-base-finetuned-stackoverflow-ner")


In [ ]:
#loading the data
df = pd.read_json("/content/datafinal.json")
print(df.info())

In [ ]:
#Inferencing the model to classify tokens and filters only codeblocks from the text
res = []
code_blocks = []
#Start Detect using default model without training. Only Inference
for text in tqdm(df['Text']):
    #print(type(pipe(text)))
    ouptut_res = pipe(text)
    res.append(ouptut_res)
    code_block = ''
    for result in ouptut_res:
      #print(result)
      if "Code_Block" in  result['entity'] or "Value" in result['entity']:
        code_block += result['word'].replace("Ġ"," ").replace("Ċ", " ")

    code_blocks.append(code_block)

In [ ]:
df['Result_li'] = res
df['Code_block_NER'] = code_blocks

In [ ]:
op = df.copy()

In [ ]:
%%time
code_li_new = []
for r, txt in zip(op['Result_li'], op['Text']):
    #print("*"*100)
    #print(txt)
    #print("*"*100)
    code = ""
    res = ast.literal_eval(r)
    if len(res) > 0:
        temp = pd.DataFrame(res)
        code_start_pos = temp[temp['entity'].str.contains("Code_Block")]['start'].min()
        code_end_pos   = temp[temp['entity'].str.contains("Code_Block|Class")]['end'].max()
        #print(code_end_pos, code_start_pos)
        if code_end_pos >= 0 and code_start_pos >= 0:
            #code =  txt[code_start_pos:code_end_pos]
            #code = code.replace("\n", " ").replace("\t", " ")
            
            for rs in res:
                if rs['start'] >= code_start_pos and rs['end'] <= code_end_pos:
                    code = code + rs['word'].replace("Ġ", " ").replace("Ċ", " ").replace("ĉ", " ")
    else:
        code = ""
    code_li_new.append(code)
    #print(code)
    #print("_"*100)

In [ ]:
mlb = MultiLabelBinarizer()
df["CodeList"].fillna("",inplace=True)
s1 = df["CodeList"]
t1 = mlb.fit_transform(s1)

In [ ]:
op['CodeList'].fillna('', inplace=True)
op["Code_block_NER_new"] = [nr if cl=="" else cl for cl, nr in zip(op['CodeList'], op['Code_block_NER'])] # op['Code_block_NER']
op["Code_block_NER_new"].fillna("",inplace=True)


In [ ]:
#Submission file Creation

pd.DataFrame(mlb.transform(op['Code_block_NER_new'])).to_csv("submision_base_bert_ner_result.csv", index=False)